# CM[8] Kaggle Competition Group38

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
import scipy

In [6]:
df = pd.read_csv("dkmacovid_kaggletest_features.csv")
df_train = pd.read_csv('cleaned_normalized_coviddata.csv')
X_train = df_train.iloc[:,2:-3]
y_train = df_train.iloc[:,-3:].astype('int')
df.head(5)

,Id,Day,State ID,State,Lat,Long_,Active,Incident_Rate,Total_Test_Results,Case_Fatality_Ratio,Testing_Rate,Resident Population 2020 Census,Population Density 2020 Census,Density Rank 2020 Census,SexRatio
0,0,2,14,Illinois,40.3495,-88.9861,957138,7697.015291,13436652,1.867428,106035.6834,"12,812,508",230.8,14,97
1,5,3,14,Illinois,40.3495,-88.9861,961499,7732.282519,13482117,1.869933,106394.4716,"12,812,508",230.8,14,97
2,10,4,14,Illinois,40.3495,-88.9861,966468,7772.205747,13530371,1.869466,106775.2693,"12,812,508",230.8,14,97
3,15,5,14,Illinois,40.3495,-88.9861,973157,7826.175891,13617454,1.871700,107462.4870,"12,812,508",230.8,14,97
4,20,6,14,Illinois,40.3495,-88.9861,980553,7885.906848,13698428,1.874835,108101.4954,"12,812,508",230.8,14,97


In [7]:
df.dtypes

Id                                   int64
Day                                  int64
State ID                             int64
State                               object
Lat                                float64
Long_                              float64
Active                               int64
Incident_Rate                      float64
Total_Test_Results                   int64
Case_Fatality_Ratio                float64
Testing_Rate                       float64
Resident Population 2020 Census     object
Population Density 2020 Census     float64
Density Rank 2020 Census             int64
SexRatio                             int64
dtype: object

In [8]:
df['Resident Population 2020 Census'] = df['Resident Population 2020 Census'].str.replace(',','').astype(int)

In [9]:
df.isna().sum()

Id                                 0
Day                                0
State ID                           0
State                              0
Lat                                0
Long_                              0
Active                             0
Incident_Rate                      0
Total_Test_Results                 0
Case_Fatality_Ratio                0
Testing_Rate                       0
Resident Population 2020 Census    0
Population Density 2020 Census     0
Density Rank 2020 Census           0
SexRatio                           0
dtype: int64

In [10]:
(df.iloc[:,4:]<0).sum()

Lat                                  0
Long_                              150
Active                               0
Incident_Rate                        0
Total_Test_Results                   0
Case_Fatality_Ratio                  0
Testing_Rate                         0
Resident Population 2020 Census      0
Population Density 2020 Census       0
Density Rank 2020 Census             0
SexRatio                             0
dtype: int64

In [11]:
df_gstate = df.groupby('State')

In [33]:
for key,value in df_gstate:
    groups = df_gstate.get_group(key)
    temp = groups.iloc[:,4:]
    for columns in temp:
        Q1 = np.percentile(temp[columns],25)
        Q3 = np.percentile(temp[columns],75)
        IQR = Q3 - Q1
        right_limit = Q3 + 1.5*IQR
        left_limit = Q1 - 1.5*IQR
        outlier_right_index = groups[groups[columns] > right_limit][columns].index
        outlier_left_index = groups[groups[columns] < left_limit][columns].index
        n_outliers = len(outlier_right_index) + len(outlier_left_index)
        if(n_outliers > 0):
            print(key,columns,n_outliers)
            df.loc[outlier_right_index,columns] = right_limit
            df.loc[outlier_left_index,columns] = left_limit

In [13]:
X_test = df.iloc[:,4:]

In [14]:
X_test = (X_test - X_test.mean())/ X_test.std()

In [15]:
X_test

,Lat,Long_,Active,Incident_Rate,Total_Test_Results,Case_Fatality_Ratio,Testing_Rate,Resident Population 2020 Census,Population Density 2020 Census,Density Rank 2020 Census,SexRatio
0,0.185758,0.685045,1.624921,-0.034555,0.871610,1.714293,1.248503,0.093862,1.600344,-1.387770,-0.557150
1,0.185758,0.685045,1.636807,-0.019067,0.879419,1.725587,1.261108,0.093862,1.600344,-1.387770,-0.557150
2,0.185758,0.685045,1.650350,-0.001535,0.887708,1.723482,1.274487,0.093862,1.600344,-1.387770,-0.557150
3,0.185758,0.685045,1.668580,0.022167,0.902667,1.733551,1.298632,0.093862,1.600344,-1.387770,-0.557150
4,0.185758,0.685045,1.688737,0.048398,0.916576,1.747687,1.321083,0.093862,1.600344,-1.387770,-0.557150
...,...,...,...,...,...,...,...,...,...,...,...
145,1.449287,-1.903393,-0.163129,-1.654112,-0.672606,-0.486315,-0.425243,-0.463803,-0.351668,-0.102798,1.114301
146,1.449287,-1.903393,-0.156348,-1.639579,-0.668592,-0.490357,-0.414460,-0.463803,-0.351668,-0.102798,1.114301
147,1.449287,-1.903393,-0.151034,-1.628091,-0.664255,-0.469195,-0.402811,-0.463803,-0.351668,-0.102798,1.114301
148,1.449287,-1.903393,-0.146139,-1.617734,-0.659315,-0.505139,-0.389542,-0.463803,-0.351668,-0.102798,1.114301


## Preprocessing Completed

# Label Recovered

In [46]:
classifier_RandomForest_recovered = RandomForestClassifier(max_depth = 3,n_estimators = 150, random_state=0)

In [47]:
classifier_RandomForest_recovered.fit(X_train,y_train.loc[:,'Recovered'])

RandomForestClassifier(max_depth=3, n_estimators=150, random_state=0)

In [48]:
y_pred_recovered = classifier_RandomForest_recovered.predict(X_test)

In [49]:
y_pred_recovered

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Label: Deaths

In [50]:
classifier_RandomForest_deaths = RandomForestClassifier(max_depth = 5,n_estimators = 150, random_state=0)

In [51]:
classifier_RandomForest_deaths.fit(X_train,y_train.loc[:,'Deaths'])

RandomForestClassifier(max_depth=5, n_estimators=150, random_state=0)

In [52]:
y_pred_deaths = classifier_RandomForest_deaths.predict(X_test)

In [53]:
y_pred_deaths

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# Label : Confirmed

In [54]:
classifier_RandomForest_confirmed = RandomForestClassifier(max_depth = 5,n_estimators = 150, random_state=0)

In [55]:
classifier_RandomForest_confirmed.fit(X_train,y_train.loc[:,'Confirmed'])

RandomForestClassifier(max_depth=5, n_estimators=150, random_state=0)

In [56]:
y_pred_confirmed = classifier_RandomForest_confirmed.predict(X_test)

In [57]:
y_pred_confirmed.shape

(150,)

In [58]:
new_df = pd.DataFrame()

In [59]:
new_df['Id'] = df.loc[:,'Id']
new_df['Confirmed'] = y_pred_confirmed
new_df['Deaths'] = y_pred_deaths
new_df['Recovered'] = y_pred_recovered

In [60]:
new_df

,Id,Confirmed,Deaths,Recovered
0,0,1,1,1
1,5,1,1,1
2,10,1,1,1
3,15,1,1,1
4,20,1,1,1
...,...,...,...,...
145,129,1,1,0
146,134,1,1,0
147,139,1,1,0
148,144,1,1,0


In [61]:
new_df.to_csv("Kagglepred_new.csv",index = False)